In [5]:
import numpy as np
import time
import math 
import random 
import copy

In [7]:

def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    N, D, A, B = map(int, lines[0].split())
    dayoff = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        numbers = list(map(int, lines[i].split()))
        for j in range(0, len(numbers) - 1):
            dayoff[i][numbers[j]] = 1
    return N, D, A, B, dayoff

In [9]:
def initialize_random_solution(N, D, dayoff, A, B):
    X = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        for j in range(1, D+1):
            if dayoff[i][j] == 1:
                X[i][j] = 0
    for i in range(1, N+1):
        for j in range(1, D+1):
            if dayoff[i][j]:
                continue
            if j> 1 and X[i][j - 1] == 4:
                X[i][j] = 0
            else: 
                X[i][j] = np.random.randint(0, 4)
    for d in range(1, D+1):
        for shift in range(1, 5):
            count = np.sum(X[:, d] == shift)
            while count < A: 
                available = [i for i in range(1, N+1) if X[i][d] == 0 and dayoff[i][d] == 0 and X[i][d-1] != 4]
                if not available:
                    break
                i = np.random.choice(available)
                X[i][d] = shift
                count += 1
            while count > B:
                assigned = np.where(X[:, d] == shift)[0]
                if not assigned.size:
                    break
                i = np.random.choice(assigned)
                X[i][d] = 0
                count -= 1
    return X


In [11]:
def evaluate_fitness(X, N, D, A, B, dayoff):
    penalty = 0
    # check vi pham ngay nghi
    for i in range(1, N + 1):
        for d in range(1, D +1):
            if dayoff[i][d] and X[i][d] != 0:
                penalty -= 1000

    # check vi pham so luong nguoi lam
    for d in range(1, D +1):
        for shift in range(1, 5):
            count = np.sum(X[:, d] == shift)
            if count < A:
                penalty -= 100 * (A - count)
            elif count > B:
                penalty -= 100 * (count - B)

    # check vi pham ca dem 
    for i in range(1, N + 1):
        for d in range(1, D):
            if X[i][d] == 4 and X[i][d + 1] != 0:
                penalty -= 1000

    max_night_shift = 0
    for i in range(1, N + 1):
        count = np.sum(X[i] == 4)
        if count > max_night_shift:
            max_night_shift = count
    return -max_night_shift + penalty

In [13]:
def max_shift_night(X,N):
    max_num =0 
    for i in range(1,N+1):
        count=np.sum(X[i]==4)
        if count > max_num:
            max_num = count
    return max_num

In [15]:
def get_neighbor(X, N, D, A, B, dayoff):
    X_new = np.copy(X)
    i = np.random.randint(1, N + 1)
    d = np.random.randint(1, D + 1)

    if dayoff[i][d] == 1: 
        return X_new
    if d > 1 and X[i][d - 1] == 4:
        return X_new
    else: 
        new_shift = np.random.randint(0, 5)
        while new_shift == X[i][d]:
            new_shift = np.random.randint(0, 5)
        X_new[i][d] = new_shift
        if new_shift == 4 and d< D:
            X_new[i][d + 1] = 0
            X_new[i][d+1] = 0
    
    #check vi pham ngay nghi
    for i in range(1, N + 1):
        for d in range(1, D + 1):
            if dayoff[i][d] == 1:
                X_new[i][d] = 0

    #check vi pham ca dem
    for i in range(1, N + 1):
        for d in range(1, D):
            if X_new[i][d] == 4 and X_new[i][d + 1] != 0:
                X_new[i][d+1] = 0


    # toi thieu hoa so nguoi lam moi ca
    for i in range(1, D + 1):
        for shift in range(1, 5):
            count = np.sum(X_new[:, i] == shift)
            while A < count < B:
                assigned = np.where(X_new[:, i] == shift)[0]
                if not assigned.size:
                    break
                j = np.random.choice(assigned)
                X_new[j][i] = 0
                count -= 1
    #check vi pham so nguoi lam
    for d in range(1, D + 1):
        for shift in range(1, 5):
            count = np.sum(X_new[:, d] == shift)
            while count < A: 
                available = [i for i in range(1, N + 1) if X_new[i][d] == 0 and dayoff[i][d] == 0 and X_new[i][d-1] != 4]
                if not available:
                    break
                i = np.random.choice(available)
                X_new[i][d] = shift
                count += 1
            while count > B:
                assigned = np.where(X_new[:, d] == shift)[0]
                if not assigned.size:
                    break
                i = np.random.choice(assigned)
                X_new[i][d] = 0
                count -= 1
    for i in range(1, N + 1):
        for d in range(1, D):
            if dayoff[i][d] == 4 and X_new[i][d + 1] != 0:
                X_new[i][d +1] = 0 
    return X_new
    
        
        

In [17]:
def simulated_annealing(N,D,A,B, dayoff, max_iterations = 10000):
    initial_temp =100.0
    min_temp=1.0
    alpha = 0.95
    iter_per_temp=100

    # khởi tạo giải pháp
    X = initialize_random_solution(N, D, dayoff, A, B)
    current_cost = evaluate_fitness(X, N, D, A, B, dayoff)
    current_max_night = max_shift_night(X,N)

    # lưu tạm giải pháp
    best_solution = X
    best_cost = current_cost
    best_max_night = current_max_night 
    temp = initial_temp 
    iteration =0 
    while temp > min_temp and iteration < max_iterations :
        for i in range(iter_per_temp):
            neighbor = get_neighbor(X, N, D, A, B, dayoff)
            neighbor_cost = evaluate_fitness(neighbor,N, D, A, B, dayoff)
            neighbor_max_night_shift = max_shift_night(neighbor,N)
            delta_cost = neighbor_cost - current_cost
            if delta_cost <= 0 or random.random() <math.exp(-delta_cost / temp):
                current_solution = neighbor 
                current_cost = neighbor_cost 
                current_max_night = neighbor_max_night_shift
                if current_cost < best_cost :
                    best_solution = current_solution 
                    best_cost = current_cost
                    best_max_night = current_max_night 
                    
                    
        temp= temp*alpha
        iteration += iter_per_temp
    print(f"Nhiệt độ: {temp:.4f}, Chi phí tốt nhất: {best_cost}, Max đêm: {best_max_night}")
    return best_solution, best_cost, best_max_night


In [19]:
# if __name__ == "__main__":
#     filename = 'Testcase/tc'
#     output = 'Output/output'
#     list_test = [1, 2,6]
#     for tc in list_test:
#         start_time = time.time()
#         input_file = f"{filename}{tc}.txt"
#         output_file = f"{output}{tc}.txt"

#         N, D, A, B, dayoff = read_input_from_file(input_file)
#         best_solution,best_cost, best_max_night = simulated_annealing(N, D, A, B, dayoff)
#         best_fitness = evaluate_fitness(best_solution, N, D, A, B, dayoff)
#         end_time = time.time()
#         print(f"Test case {tc} completed in {end_time - start_time:.2f} seconds.")

In [ ]:
if __name__ == "__main__":
    filename = 'Testcase/tc'
    output = 'Output/SA/output'
    list_test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    for tc in list_test:
        start_time = time.time()
        input_file = f"{filename}{tc}.txt"
        output_file = f"{output}{tc}.txt"

        N, D, A, B, dayoff = read_input_from_file(input_file)
        best_solution,best_cost, best_max_night = simulated_annealing(N, D, A, B, dayoff)
        best_fitness = evaluate_fitness(best_solution, N, D, A, B, dayoff)
        with open(output_file, 'w') as file:
            for i in range(1, N + 1):
                file.write(' '.join([str(int(best_solution[i][d])) for d in range(1, D + 1)]) + '\n')
            end_time = time.time()
            file.write("Execution time: {:.2f} seconds\n".format(end_time - start_time))
        print(f"Test case {tc} completed in {end_time - start_time:.2f} seconds.")

Nhiệt độ: 0.9888, Chi phí tốt nhất: -3102, Max đêm: 2
Test case 1 completed in 5.12 seconds.
